TODO

fare review del codice, pulirlo sitemarlo

introdurre l'implementazione e spiegarne le scelte

codice fino al training

# Midterm 3, Assignment 1 - Gaetano Barresi [579102]

A Denoising Autoencoder (DAE) is a neural network that learns robust latent representations by reconstructing clean inputs from corrupted data. Unlike standard autoencoders that simply reconstruct the original inputs, a DAE must remove noise from artificially corrupted inputs and discover stable features that capture the true data distribution. The goal of this work is to train two versions of a DAE on the CIFAR10 dataset, one using dense layers and one using convolutional layers and to show an accuracy comparison between them.

The architecture of a DAE has the following form:

$$
\
[Input] → [Corruption] → [Encoder] → [Latent Code] → [Decoder] → [Reconstruction]
\
$$

$[Input]$ is self-explanatory. $[Corruption]$ is the stage where we inject artificial noise to the input through some noise process $C(\hat{x} ∣ x)$ and obtain $\hat{x}$, a corrupted version of the original input. In this work the corruption process is done via standard Gaussian noise, so:

$$
\
\hat{x} = x + ϵ, ϵ ∼ 𝒩(0, σ^2).
\
$$

$[Encoder]$ $f()$ maps noisy input $\hat{x}$ to latent representation $z$:

$$
\
z = f(\hat{x}) ∈ ℝᵈ,
\
$$

where $d << input\_dim$. Learned $[Latent Code]$ $z$ should be robust to partial destruction of the input.
$[Decoder]$ $g()$ reconstructs clean input from $z$:

$$
\
x' = g(z) ≈ x,
\
$$

where $x'$ is our $[Reconstruction]$.

The "magic" of DAEs lies in their bottleneck architecture. As data passes through the encoder, the network gradually compresses the input while stripping away noise and preserving only the most essential features. This forced dimensionality reduction creates a distilled latent representation containing only the core patterns needed for reconstruction. The decoder then reverses this process, carefully rebuilding the clean input from these compressed features layer by layer. By training on noisy-clean pairs, the network learns to discard random corruptions during compression while maintaining the structural integrity needed for accurate reconstruction.

More formally, the DAE is trained to minimize the loss function

$$
\
L = (x, g(f(\hat{x}))),
\
$$

or, using a probabilistic interpretation, the DAE learns the denoising distribuition

$$
\
P(x|\hat{x})
\
$$

by minimizing

$$
\
-log P(x|z = f(\hat{x})).
\
$$

After this brief introduction, we proceed with the implementation. First, we need to load and preprocess the dataset. We normalize the CIFAR-10 data to the range [-1, 1] to achieve several benefits such as: compatibility with the tanh activation, effective noise handling due to the symmetric range that ensures Gaussian noise is equally well-handled in both positive and negative directions, gradient flow optimization, numerical stability.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# transform for CIFAR-10, normalizing to [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Range: [-1, 1]
])
# Load TR set
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
# Load TS set
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_data, batch_size=100, shuffle=False)

print("Data loaded successfully.")

Next step is to define the DAE class